In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
#matplotlib inline

In [ ]:
!pip install utils

In [ ]:
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56439 sha256=3c04a22e52be49e62efbb0419a1ce501a307edf54f95fa5036f784e1aff8bf16
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
from utils import *

from keras.models import load_model
from sklearn.datasets import load_files
#from keras.utils import np_utils
from keras.utils import to_categorical
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

In [ ]:
# To mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/deep drive/csv'

/content/gdrive/My Drive/deep drive/csv


In [ ]:
# train = pd.read_csv('multi_label_train.csv')    # reading the csv file
# train.head()      # printing first five rows of the file
import pandas as pd
train = pd.read_csv('multi_label_train.csv')
train = train.drop(train[train.Id == 'tt0086425'].index)
train.to_csv('multi_label_train.csv', index=False)
train.head()

,Id,Genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,N/A,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
0,tt0085549,"['Drama', 'Romance', 'Music']",0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,tt0086465,['Comedy'],0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt0086567,"['Sci-Fi', 'Thriller']",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,tt0086034,"['Action', 'Adventure', 'Thriller']",1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,tt0086383,"['Action', 'Thriller']",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
train.columns

Index(['Id', 'Genre', 'Action', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'N/A', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [ ]:

pip install tqdm

In [ ]:
from tqdm import tqdm
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('Image/Images/'+train['Id'][i]+'.jpg',target_size=(128,128,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

 27%|██▋       | 1941/7253 [10:02<23:57,  3.70it/s]

In [ ]:
 X.shape

In [ ]:
plt.imshow(X[4])

train['Genre'][4]

In [ ]:
y = np.array(train.drop(['Id', 'Genre'],axis=1))
y.shape
print(y)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(128,128,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='sigmoid'))

model.summary()

In [ ]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=64)


In [ ]:
img = image.load_img('Image/Images/tt0085970.jpg',target_size=(128,128,3))
img = image.img_to_array(img)
img = img/255

classes = np.array(train.columns[2:])
proba = model.predict(img.reshape(1,128,128,3))
top_3 = np.argsort(proba[0.5])[:-4:-1]
for i in range(3):
    print("{}".format(classes[top_3[i]])+" ({:.3})".format(proba[0][top_3[i]]))
plt.imshow(img)


In [ ]:
#If imagenet weights are being loaded,
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
#base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (128,128,3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(25, activation= 'softmax')(x)
model1 = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
from keras.optimizers import SGD, Adam
# sgd = SGD(learning_rate =lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.01)
model1.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model1.fit(X_train, y_train, epochs = 10, batch_size = 64)

Epoch 1/10
102/102 [==============================] - 61s 249ms/step - loss: 25.9714 - accuracy: 0.1989
Epoch 2/10
102/102 [==============================] - 23s 224ms/step - loss: 46.2612 - accuracy: 0.1811
Epoch 3/10
102/102 [==============================] - 23s 221ms/step - loss: 71.5255 - accuracy: 0.1740
Epoch 4/10
102/102 [==============================] - 23s 221ms/step - loss: 105.6646 - accuracy: 0.1966
Epoch 5/10
102/102 [==============================] - 23s 222ms/step - loss: 151.4666 - accuracy: 0.1946
Epoch 6/10
102/102 [==============================] - 23s 222ms/step - loss: 200.0793 - accuracy: 0.1786
Epoch 7/10
102/102 [==============================] - 23s 223ms/step - loss: 240.4937 - accuracy: 0.1739
Epoch 8/10
102/102 [==============================] - 23s 221ms/step - loss: 299.2784 - accuracy: 0.1730
Epoch 9/10
102/102 [==============================] - 23s 222ms/step - loss: 375.1479 - accuracy: 0.1796
Epoch 10/10
102/102 [==============================] - 23s